<img src="https://github.com/Microsoft/sqlworkshops/blob/master/graphics/solutions-microsoft-logo-small.png?raw=true" alt="Microsoft">
<br>

# **SQL Server 2019 big data cluster Tutorial**
## **Monitoring and troubleshoot SQL Server Big Data Clusters**

In this tutorial, you'll learn how to view in-depth details of a pod or other Kubernetes artifacts that are located in the big data cluster. This article also covers common tasks, such as copying files to or from a container running one of the SQL Server big data cluster services.



## **Get status of pods**
You can use the kubectl get pods command to get the status of pods in the cluster for either all namespaces or the big data cluster namespace. The following sections show examples of both.

### **Show status of all pods in the Kubernetes cluster**

Run the following command to get all the pods and their statuses, including the pods that are part of the namespace that SQL Server big data cluster pods are created in:

In [ ]:
kubectl get pods --all-namespaces

### **Show status of all pods in the SQL Server big data cluster**

Use the -n parameter to specify a specific namespace. Note that SQL Server big data cluster pods are created in a new namespace created at cluster bootstrap time based on the cluster name specified in the deployment configuration file. The default name, mssql-cluster, is used here.

In [2]:
kubectl get pods -n mssql-cluster

NAME              READY   STATUS    RESTARTS   AGE
app2-4cjvg        1/1     Running   0          10d
app2-5pxvr        1/1     Running   0          10d
appproxy-c6qt5    2/2     Running   0          63d
compute-0-0       3/3     Running   0          63d
control-6g7dz     0/3     Evicted   0          21d
control-6j595     3/3     Running   0          8d
control-fdgz4     0/3     Evicted   0          34d
control-lrnbw     0/3     Evicted   0          63d
control-vrxwg     0/3     Evicted   0          49d
controldb-0       2/2     Running   0          63d
controlwd-27hls   1/1     Running   0          63d
data-0-0          3/3     Running   0          63d
data-0-1          3/3     Running   0          63d
gateway-0         2/2     Running   0          63d
logsdb-0          1/1     Running   4          63d
logsui-dg5s4      1/1     Running   0          63d
master-0          3/3     Running   0          63d
metricsdb-0       1/1     Running   0          63d
metricsdc-pvk7l   1/1     Runnin

You should see output similar to the following list for a running big data cluster:

```
PS C:\> kubectl get pods -n mssql-cluster
NAME              READY   STATUS    RESTARTS   AGE
appproxy-f2qqt    2/2     Running   0          110m
compute-0-0       3/3     Running   0          110m
control-zlncl     4/4     Running   0          118m
data-0-0          3/3     Running   0          110m
data-0-1          3/3     Running   0          110m
gateway-0         2/2     Running   0          109m
logsdb-0          1/1     Running   0          112m
logsui-jtdnv      1/1     Running   0          112m
master-0          7/7     Running   0          110m
metricsdb-0       1/1     Running   0          112m
metricsdc-shv2f   1/1     Running   0          112m
metricsui-9bcj7   1/1     Running   0          112m
mgmtproxy-x6gcs   2/2     Running   0          112m
nmnode-0-0        1/1     Running   0          110m
storage-0-0       7/7     Running   0          110m
storage-0-1       7/7     Running   0          110m
```

## **Get pod details**

Run the following command to get a detailed description of a specific pod in JSON format output. It includes details, such as the current Kubernetes node that the pod is placed on, the containers running within the pod, and the image used to bootstrap the containers. It also shows other details, such as labels, status, and persisted volumes claims that are associated with the pod.

In [ ]:
kubectl describe pod  <pod_name> -n <namespace_name>

The following example shows the details for the master-0 pod in a big data cluster named mssql-cluster:

In [3]:
kubectl describe pod master-0 -n mssql-cluster

Name:           master-0
Namespace:      mssql-cluster
Priority:       0
Node:           aks-agentpool-51355285-vmss000000/10.240.0.4
Start Time:     Mon, 30 Dec 2019 19:53:14 +0800
Labels:         MSSQL_CLUSTER=mssql-cluster
                app=master
                controller-revision-hash=master-5879795dc9
                mssql.microsoft.com/sql-instance=master
                plane=data
                role=master-pool
                statefulset.kubernetes.io/pod-name=master-0
                type=sqlservr
Annotations:    <none>
Status:         Running
IP:             10.244.0.20
IPs:            <none>
Controlled By:  StatefulSet/master
Containers:
  mssql-server:
    Container ID:   docker://1e100b62cfbf1c50809ab44e5da73ee3ca01810f09a63aa312bd3ee6289bdc80
    Image:          mcr.microsoft.com/mssql/bdc/mssql-server-data:2019-GDR1-ubuntu-16.04
    Image ID:       docker-pullable://mcr.microsoft.com/mssql/bdc/mssql-server-data@sha256:e4f91e45dae393dd36289c77a0ea9d673611a7c8a934e60

If any errors have occurred, you can sometimes see the error in the recent events for the pod.

## **Get pod logs**

You can retrieve the logs for containers running in a pod. The following command retrieves the logs for all containers running in the pod named master-0 and outputs them to a file name master-0-pod-logs.txt:

In [ ]:
kubectl logs master-0 --all-containers=true -n mssql-cluster > ~/temp/master-0-pod-logs.txt

## **Get status of services**

Run the following command to get details for the big data cluster services. These details include their type and the IPs associated with respective services and ports. Note that SQL Server big data cluster services are created in a new namespace created at cluster bootstrap time based on the cluster name specified in deployment configuration file.

In [ ]:
kubectl get svc -n <namespace_name>

The following example shows the status for services in a big data cluster named mssql-cluster:

In [ ]:
kubectl get svc -n mssql-cluster

| Service                 | Description                                                                 |
|-------------------------|-----------------------------------------------------------------------------|
| master-svc-external     | Provides access to the master instance. (EXTERNAL-IP,31433 and the SA user) |
| controller-svc-external | Supports tools and clients that manage the cluster.                         |
| gateway-svc-external    | Provides access to the HDFS/Spark gateway. (EXTERNAL-IP and the root user)  |
| appproxy-svc-external   | Support application deployment scenarios.                                   |

## **Get service details**

Run this command to get a detailed description of a service in JSON format output. It will include details like labels, selector, IP, external-IP (if the service is of LoadBalancer type), port, etc.

In [ ]:
kubectl describe service <service_name> -n <namespace_name>

The following example retrieves details for the master-svc-external service:

In [ ]:
kubectl describe service master-svc-external -n mssql-cluster

## **Copy files**

If you need to copy files from the container to your local machine, use kubectl cp command with the following syntax:

In [ ]:
kubectl cp <pod_name>:<source_file_path> -c <container_name> -n <namespace_name> <target_local_file_path>

Similarly, you can use kubectl cp to copy files from the local machine into a specific container.

In [ ]:
kubectl cp <source_local_file_path> <pod_name>:<target_container_path> -c <container_name>  -n <namespace_name>

### **Copy files from a container**

The following example copies the SQL Server log files from the container to the ~/temp/sqlserverlogs path on the local machine (in this example the local machine is a Linux client):

In [ ]:
kubectl cp master-0:/var/opt/mssql/log -c mssql-server -n mssql-cluster ~/tmp/sqlserverlogs

### **Copy files into container**

The following example copies the AdventureWorks2016CTP3.bak file from the local machine to the SQL Server master instance container (mssql-server) in the master-0 pod. The file is copied to the /tmp directory in the container.

In [ ]:
kubectl cp ~/temp/ master-0:/tmp -c mssql-server -n mssql-cluster

## **Force delete a pod**

It is not recommended to force-delete a pod. But for testing availability, resiliency, or data persistence, you can delete a pod to simulate a pod failure with the kubectl delete pods command.

In [ ]:
kubectl delete pods <pod_name> -n <namespace_name> --grace-period=0 --force

The following example deletes the storage pool pod, storage-0-0:

In [ ]:
kubectl delete pods storage-0-0 -n mssql-cluster --grace-period=0 --force

## **Get pod IP**

For troubleshooting purposes, you might need to get the IP of the node a pod is currently running on. To get the IP address, use the kubectl get pods command with the following syntax:

In [ ]:
kubectl get pods <pod_name> -o yaml -n <namespace_name> | grep hostIP

The following example gets the IP address of the node that the master-0 pod is running on:

In [ ]:
kubectl get pods master-0 -o yaml -n mssql-cluster | grep hostIP

## **Kubernetes dashboard**
You can launch the Kubernetes dashboard for additional information about the cluster. The following sections explain how to launch the dashboard for Kubernetes on AKS and for Kubernetes bootstrapped using kubeadm.

### **Start dashboard when cluster is running in AKS**

To launch the Kubernetes dashboard run:

In [ ]:
az aks browse --resource-group <azure_resource_group> --name <aks_cluster_name>

>If you get the following error: Unable to listen on port 8001: All listeners failed to create with the following errors: Unable to create listener: Error listen tcp4 127.0.0.1:8001: >bind: Only one usage of each socket address (protocol/network address/port) is normally permitted. Unable to create listener: Error listen tcp6: address [[::1]]:8001: missing port in >address error: Unable to listen on any of the requested ports: [{8001 9090}], make sure you did not start the dashboard already from another window.

When you launch the dashboard on your browser, you might get permission warnings due to RBAC being enabled by default in AKS clusters, and the service account used by the dashboard does not have enough permissions to access all resources (for example, pods is forbidden: User "system:serviceaccount:kube-system:kubernetes-dashboard" cannot list pods in the namespace "default"). Run the following command to give the necessary permissions to kubernetes-dashboard, and then restart the dashboard:

### **Start dashboard when Kubernetes cluster is bootstrapped using kubeadm**

For detailed instructions how to deploy and configure the dashboard in your Kubernetes cluster, see the Kubernetes documentation. To launch the Kubernetes dashboard, run this command:

In [ ]:
kubectl proxy